# NYC Parking Tickets Case Study 

In [1]:
##Creating spark object
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("parking").getOrCreate()

In [2]:
## Reading the Parking data
parking = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/common_folder/nyc_parking/Parking_Violations_Issued_-_Fiscal_Year_2017.csv")

In [3]:
parking.show(2)

+--------------+--------+------------------+-------------------+--------------+-----------------+------------+------------------+---------------+--------------+
|Summons Number|Plate ID|Registration State|         Issue Date|Violation Code|Vehicle Body Type|Vehicle Make|Violation Precinct|Issuer Precinct|Violation Time|
+--------------+--------+------------------+-------------------+--------------+-----------------+------------+------------------+---------------+--------------+
|    5092469481| GZH7067|                NY|2016-07-10 00:00:00|             7|             SUBN|       TOYOT|                 0|              0|         0143A|
|    5092451658| GZH7067|                NY|2016-07-08 00:00:00|             7|             SUBN|       TOYOT|                 0|              0|         0400P|
+--------------+--------+------------------+-------------------+--------------+-----------------+------------+------------------+---------------+--------------+
only showing top 2 rows



In [4]:
#Importing all sql functions
from pyspark.sql.functions import *

In [5]:
#Converting Issue Date column from String to Date type and extracting year
parking=parking.withColumn("Issue Date",to_date("`Issue Date`", "yyyy-MM-dd"))
parking=parking.withColumn("year_issue",year("`Issue Date`"))

In [6]:
#Creating SQL Temp view from parking dataframe
parking.createOrReplaceTempView("parking")

In [7]:
#Checking if year is correctly extracted
spark.sql("select `Issue Date`, year_issue from parking").show(5)

+----------+----------+
|Issue Date|year_issue|
+----------+----------+
|2016-07-10|      2016|
|2016-07-08|      2016|
|2016-08-23|      2016|
|2017-06-14|      2017|
|2016-11-21|      2016|
+----------+----------+
only showing top 5 rows



In [8]:
#Let's check total no. of rows
parking.count()

10803028

In [9]:
#Checking columns in Parking
parking.printSchema()

root
 |-- Summons Number: long (nullable = true)
 |-- Plate ID: string (nullable = true)
 |-- Registration State: string (nullable = true)
 |-- Issue Date: date (nullable = true)
 |-- Violation Code: integer (nullable = true)
 |-- Vehicle Body Type: string (nullable = true)
 |-- Vehicle Make: string (nullable = true)
 |-- Violation Precinct: integer (nullable = true)
 |-- Issuer Precinct: integer (nullable = true)
 |-- Violation Time: string (nullable = true)
 |-- year_issue: integer (nullable = true)



In [10]:
#Creating Dataframe where year of Issue date is 2017
parking1 = parking.where(col("year_issue") == 2017)

In [11]:
#Creating SQL Temp view from parking1 dataframe
parking1.createOrReplaceTempView("parking1")

## Examine the data

### Q1. Find the total number of tickets for the year.

In [12]:
#Total No. of tickets in year 2017
spark.sql("select count(`Summons Number`) as count from parking1").show()

+-------+
|  count|
+-------+
|5431918|
+-------+



### Q2. Find out the number of unique states from where the cars that got parking tickets came.

In [13]:
#No. of Unique states
spark.sql("select count(distinct `Registration State`) from parking1").show()

+----------------------------------+
|count(DISTINCT Registration State)|
+----------------------------------+
|                                65|
+----------------------------------+



In [14]:
#Checking the top 5 states having highest no. of tickets
spark.sql("Select *,dense_rank() over (order by a.count desc) as rank from (Select count(*) as count, `Registration State` from parking1 group by `Registration State`) a").show(5)


+-------+------------------+----+
|  count|Registration State|rank|
+-------+------------------+----+
|4273951|                NY|   1|
| 475825|                NJ|   2|
| 140286|                PA|   3|
|  70403|                CT|   4|
|  69468|                FL|   5|
+-------+------------------+----+
only showing top 5 rows



In [15]:
#Replacing '99' with Registration state having maximum entries i.e. NY
parking2 = parking1.replace(["99"], ["NY"], ["Registration State"])

In [16]:
#Checking count of Registration state with value '99'
parking2.createOrReplaceTempView("parking2")

spark.sql("select count(*) from parking2 where `Registration State` = '99'").show()

+--------+
|count(1)|
+--------+
|       0|
+--------+



In [17]:
#Checking no. of unique states again
spark.sql("select count(distinct `Registration State`) from parking2").show()

+----------------------------------+
|count(DISTINCT Registration State)|
+----------------------------------+
|                                64|
+----------------------------------+



## Aggregation tasks

### Q1. How often does each violation code occur? Display the frequency of the top five violation codes.

In [18]:
#Checking the top 5 violation codes
#using sql query
spark.sql("Select *,dense_rank() over (order by a.count desc) as rank from (Select count(*) as count, `Violation Code` from parking2 group by `Violation Code`) a").show(5)


+------+--------------+----+
| count|Violation Code|rank|
+------+--------------+----+
|768087|            21|   1|
|662765|            36|   2|
|542079|            38|   3|
|476664|            14|   4|
|319646|            20|   5|
+------+--------------+----+
only showing top 5 rows



In [19]:
#using Python query
parking2.groupby("`Violation Code`").agg(count("`Violation Code`").alias("count")).sort("count", ascending=False).show(5)

+--------------+------+
|Violation Code| count|
+--------------+------+
|            21|768087|
|            36|662765|
|            38|542079|
|            14|476664|
|            20|319646|
+--------------+------+
only showing top 5 rows



### Q2. How often does each 'vehicle body type' get a parking ticket? How about the 'vehicle make'? 

In [20]:
## Top 5 vehicle body type getting max. no. of parking tickets
parking2.groupby("`Vehicle Body Type`").agg(count("`Vehicle Body Type`").alias("count")).sort("count", ascending=False).show(5)

+-----------------+-------+
|Vehicle Body Type|  count|
+-----------------+-------+
|             SUBN|1883954|
|             4DSD|1547312|
|              VAN| 724029|
|             DELV| 358984|
|              SDN| 194197|
+-----------------+-------+
only showing top 5 rows



Vehicle body type 'SUBN' got maximum tickets.

In [21]:
## Top 5 vehicle make getting max. no. of parking tickets
parking2.groupby("`Vehicle Make`").agg(count("`Vehicle Make`").alias("count")).sort("count", ascending=False).show(5)

+------------+------+
|Vehicle Make| count|
+------------+------+
|        FORD|636844|
|       TOYOT|605291|
|       HONDA|538884|
|       NISSA|462017|
|       CHEVR|356032|
+------------+------+
only showing top 5 rows



Vehicles having make type as "FORD" got maximum no. of tickets.

### Q3. A precinct is a police station that has a certain zone of the city under its command. Find the (5 highest) frequencies of tickets for each of the following: Violation Precinct, Issuer Precinct.

#### 1. Violation Precinct

In [22]:
## Top 6 Violation Precinct having max. no. of parking tickets
parking2.groupby("`Violation Precinct`").agg(count("`Violation Precinct`").alias("count")).sort("count", ascending=False).show(6)

+------------------+------+
|Violation Precinct| count|
+------------------+------+
|                 0|925596|
|                19|274445|
|                14|203553|
|                 1|174702|
|                18|169131|
|               114|147444|
+------------------+------+
only showing top 6 rows



As maximum no. of tickets are for violation precinct '0' which is an erroneous value, hence Violation Precinct '19' has max. tickets.

#### 2. Issuer Precinct

In [23]:
## Top 6 Issuer Precinct having max. no. of parking tickets
parking2.groupby("`Issuer Precinct`").agg(count("`Issuer Precinct`").alias("count")).sort("count", ascending=False).show(6)

+---------------+-------+
|Issuer Precinct|  count|
+---------------+-------+
|              0|1078406|
|             19| 266961|
|             14| 200495|
|              1| 168740|
|             18| 162994|
|            114| 144054|
+---------------+-------+
only showing top 6 rows



As maximum no. of tickets are for Issuer precinct '0' which is an erroneous value, hence Issuer Precinct '19' has max. tickets.

### Q4. Find the violation code frequency for three precincts that have issued the most number of tickets. Do these precinct zones have an exceptionally high frequency of certain violation codes? Are these codes common across precincts? 

In [24]:
##Precints that have issued most tickets are 19,14,1
spark.sql("Select * from (Select *,dense_rank() over (partition by a.`Issuer Precinct` order by a.count desc) as `rank` from (Select `Issuer Precinct`, `Violation Code`, count(*) as count from parking2 where `Issuer Precinct` in (19,14,1) group by `Issuer Precinct`, `Violation Code`)a )b where b.rank <= 5").show()


+---------------+--------------+-----+----+
|Issuer Precinct|Violation Code|count|rank|
+---------------+--------------+-----+----+
|              1|            14|38354|   1|
|              1|            16|19081|   2|
|              1|            20|15408|   3|
|              1|            46|12745|   4|
|              1|            38| 8535|   5|
|             19|            46|48445|   1|
|             19|            38|36386|   2|
|             19|            37|36056|   3|
|             19|            14|29797|   4|
|             19|            21|28415|   5|
|             14|            14|45036|   1|
|             14|            69|30464|   2|
|             14|            31|22555|   3|
|             14|            47|18364|   4|
|             14|            42|10027|   5|
+---------------+--------------+-----+----+



Violation code 14 is common across 19,14,1 precincts and has high frequency.

### Q5. (a) Find a way to deal with missing values.

In [25]:
#Checking no. of missing values in violation time column
#parking2.where(col("`Violation Time`").isNull()).count()
spark.sql("select count(*) from parking2 where `Violation Time` == 'nan'").show()
#there are 16 missing values in violation time column

+--------+
|count(1)|
+--------+
|      16|
+--------+



In [26]:
##Taking only rows where Violation time is not nan
parking3 = parking2.filter(parking2["`Violation Time`"] != 'nan')

In [27]:
#Checking again if there are any nan values in violation time
parking3.createOrReplaceTempView("parking3")
spark.sql("select count(*) from parking3 where `Violation Time` == 'nan'").show()

+--------+
|count(1)|
+--------+
|       0|
+--------+



### Q5. (b) The Violation Time field is specified in a strange format. Find a way to make this a time attribute that you can use to divide into groups.

In [28]:
##Extracting hour, time and am/pm details from Violation time column
func = udf (lambda x : x[0:2])
func1 = udf (lambda x : x[2:4])
func2 = udf (lambda x : x[-1])
parking3 = parking3.withColumn("hour_violation", func("`Violation Time`").cast(IntegerType()))
parking3 = parking3.withColumn("min_violation", func1("`Violation Time`").cast(IntegerType()))
parking3 = parking3.withColumn("AM_PM", func2("`Violation Time`"))

In [29]:
##converting hour_violation in 24hour format
hour = udf (lambda x,y : x+12 if y=='P' else x)
parking3 = parking3.withColumn("hour_violation_24", hour("hour_violation","AM_PM").cast(IntegerType()))

In [30]:
parking3.printSchema()

root
 |-- Summons Number: long (nullable = true)
 |-- Plate ID: string (nullable = true)
 |-- Registration State: string (nullable = true)
 |-- Issue Date: date (nullable = true)
 |-- Violation Code: integer (nullable = true)
 |-- Vehicle Body Type: string (nullable = true)
 |-- Vehicle Make: string (nullable = true)
 |-- Violation Precinct: integer (nullable = true)
 |-- Issuer Precinct: integer (nullable = true)
 |-- Violation Time: string (nullable = true)
 |-- year_issue: integer (nullable = true)
 |-- hour_violation: integer (nullable = true)
 |-- min_violation: integer (nullable = true)
 |-- AM_PM: string (nullable = true)
 |-- hour_violation_24: integer (nullable = true)



In [31]:
##Checking few rows
parking3.createOrReplaceTempView("parking3")
spark.sql("select hour_violation, min_violation, AM_PM, hour_violation_24 from parking3").show(5)

+--------------+-------------+-----+-----------------+
|hour_violation|min_violation|AM_PM|hour_violation_24|
+--------------+-------------+-----+-----------------+
|            11|           20|    A|               11|
|             8|           52|    P|               20|
|             0|           15|    A|                0|
|             5|           25|    A|                5|
|             2|           56|    P|               14|
+--------------+-------------+-----+-----------------+
only showing top 5 rows



### Q5. (c) Divide 24 hours into six equal discrete bins of time. Choose the intervals as you see fit. For each of these groups, find the three most commonly occurring violations.

In [32]:
##Creating 6 bins based on 24hour slot of hour_violation_24
def bins(hour_violation_24):
    if hour_violation_24 < 5:
        return "Early_Morning"
    elif 5 <= hour_violation_24 < 10:
        return "Morning"
    elif 10 <= hour_violation_24 < 17:
        return "Afternoon"
    elif 17 <= hour_violation_24 < 21:
        return "Evening"
    elif 21 <= hour_violation_24 < 24:
        return "Night"
    else:
        return "Late_Night"

bins_udf = udf(lambda x: bins(x))

parking3 = parking3.withColumn("Bins", bins_udf("hour_violation_24"))

In [33]:
##Checking 3 top most occurring violation codes for each of the bins 
parking3.createOrReplaceTempView("parking3")
spark.sql("Select * from (Select *,dense_rank() over (partition by a.Bins order by a.count desc) as `rank` from (Select Bins, `Violation Code`, count(*) as count from parking3 group by Bins, `Violation Code`)a )b where b.rank <= 3").show()


+-------------+--------------+------+----+
|         Bins|Violation Code| count|rank|
+-------------+--------------+------+----+
|      Evening|            38| 61211|   1|
|      Evening|             7| 45290|   2|
|      Evening|            14| 40214|   3|
|      Morning|            21|434446|   1|
|      Morning|            36|167209|   2|
|      Morning|            14|154628|   3|
|   Late_Night|            36|101991|   1|
|   Late_Night|            21| 72384|   2|
|   Late_Night|            38| 55967|   3|
|    Afternoon|            36|388741|   1|
|    Afternoon|            38|331487|   2|
|    Afternoon|            21|224927|   3|
|Early_Morning|            21| 35993|   1|
|Early_Morning|            40| 26662|   2|
|Early_Morning|            14| 15295|   3|
|        Night|             7| 18382|   1|
|        Night|            40| 17546|   2|
|        Night|            14| 15643|   3|
+-------------+--------------+------+----+



Violation codes 21, 14, 38 are most occurring in the bins created.

### Q5. (d) Now, try another direction. For the three most commonly occurring violation codes, find the most common time of the day.

In [34]:
##Checking 5 most commonly occurring violation codes
parking3.groupby("`Violation Code`").agg(count("`Violation Code`").alias("count")).sort("count", ascending=False).show(5)

+--------------+------+
|Violation Code| count|
+--------------+------+
|            21|768087|
|            36|662765|
|            38|542079|
|            14|476664|
|            20|319646|
+--------------+------+
only showing top 5 rows



In [35]:
##Checking most common time of day for 3 most commonly occurring violation codes i.e. 21, 36,38
spark.sql("Select * from (Select *,dense_rank() over (partition by a.`Violation Code` order by a.count desc) as `rank` from (Select `Violation Code`, Bins , count(*) as count from parking3 where `Violation Code` in (21,36,38) group by `Violation Code`, Bins)a )b where b.rank = 1").show()


+--------------+---------+------+----+
|Violation Code|     Bins| count|rank|
+--------------+---------+------+----+
|            38|Afternoon|331487|   1|
|            21|  Morning|434446|   1|
|            36|Afternoon|388741|   1|
+--------------+---------+------+----+



For Violation code 38, 36 most common time is Afternoon.

For Violation code 21 most common time is Morning.

### Q6. (a) Divide the year into a certain number of seasons, and find the frequencies of tickets for each season.

In [36]:
##Extracting month from Issue Date column
parking3=parking3.withColumn("month_issue",month("`Issue Date`"))

In [37]:
##Creating seasons based on months from Issue Date
def seasons(month_issue):
    if 3 <= month_issue < 6:
        return "Spring"
    elif 6 <= month_issue < 9:
        return "Summer"
    elif 9 <= month_issue < 12:
        return "Autumn"
    else:
        return "Winter"

seasons_udf = udf(lambda x: seasons(x))

parking3 = parking3.withColumn("Seasons", seasons_udf("month_issue"))

In [38]:
#Finding frequency of tickets in each season
parking3.groupby("Seasons").agg(count("Seasons").alias("count")).sort("count", ascending=False).show()

+-------+-------+
|Seasons|  count|
+-------+-------+
| Spring|2873371|
| Winter|1704686|
| Summer| 852866|
| Autumn|    979|
+-------+-------+



### Q6. (b) find the three most common violations for each of these seasons.

In [39]:
## Finding 3 most common violation codes for each of the seasons 
parking3.createOrReplaceTempView("parking3")
spark.sql("Select * from (Select *,dense_rank() over (partition by a.Seasons order by a.count desc) as `rank` from (Select Seasons, `Violation Code`, count(*) as count from parking3 group by Seasons, `Violation Code`)a )b where b.rank <= 3").show()


+-------+--------------+------+----+
|Seasons|Violation Code| count|rank|
+-------+--------------+------+----+
| Spring|            21|402424|   1|
| Spring|            36|344834|   2|
| Spring|            38|271167|   3|
| Summer|            21|127352|   1|
| Summer|            36| 96663|   2|
| Summer|            38| 83518|   3|
| Autumn|            46|   231|   1|
| Autumn|            21|   128|   2|
| Autumn|            40|   116|   3|
| Winter|            21|238183|   1|
| Winter|            36|221268|   2|
| Winter|            38|187386|   3|
+-------+--------------+------+----+



### Q7. (a) Find the total occurrences of the three most common violation codes.

In [40]:
##21,36, 38 are most common occurring violation codes
spark.sql("Select `Violation Code`, count(*) as count from parking3 where `Violation Code` in (21,36,38) group by `Violation Code` order by count desc").show()

+--------------+------+
|Violation Code| count|
+--------------+------+
|            21|768087|
|            36|662765|
|            38|542079|
+--------------+------+



### Q7. (b),(c) Using information of fines associated with different violation codes, find the total amount collected for the three violation codes with the maximum tickets. State the code that has the highest total collection.

Total Amount for Violation code 21 = `768087 * 55 = 42244785`

Total Amount for Violation code 36 = `662765 * 50 = 33138250`

Total Amount for Violation code 38 = `542079 * 50 = 27103950`

Violation code 21 has highest total collection.

### Q7. (d) What can you intuitively infer from these findings?

1. Fines collected for Violation code 21(Street Cleaning: No parking where parking is not allowed by sign, street marking or traffic control device) is contributing highest to the revenue of the NYC Police Department.

2. In all seasons, frequency of violation code 21 is most in comparison to other violations.

3. Max. no. of violations occurred in spring season i.e. March, April, May.

4. Vehicle body type 'SUBN' and Vehicle Make 'FORD' got maximum no. of parking tickets.

In [41]:
## Stopping the spark session
spark.stop()